<a href="https://colab.research.google.com/github/AarkeTripathi/spaceship-titanic/blob/master/space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [383]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re

In [384]:
train_data=pd.read_csv("train.csv")
train_data.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [385]:
train_data['CryoSleep']=train_data['CryoSleep'].astype(bool)
train_data['VIP']=train_data['VIP'].astype(bool)

In [386]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8693 non-null   bool   
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(3), float64(6), object(5)
memory usage: 772.6+ KB


In [387]:
f_train_data=train_data.drop(['PassengerId','Name'], axis=1)
f_train_data=f_train_data.dropna()
f_train_data.shape

(7111, 12)

In [388]:
f_train_data.reset_index(inplace=True)
f_train_data.drop('index',axis=1,inplace=True)

In [389]:
f_train_data['Deck']=f_train_data['Cabin'].apply(lambda x: x.split('/')[0])
f_train_data['Side']=f_train_data['Cabin'].apply(lambda x: x.split('/')[2])

In [390]:
f_train_data.drop(['Cabin'],axis=1,inplace=True)

In [391]:
encode=LabelEncoder()
for column in f_train_data.columns:
  if f_train_data[column].dtype!='float64':
    f_train_data[column]=encode.fit_transform(f_train_data[column])

In [392]:
# for column in f_train_data.columns:
#   if f_train_data[column].dtype=='bool':
#     f_train_data[column]=encode.fit_transform(f_train_data[column])

In [393]:
f_train_data.tail(10)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
7101,0,0,2,26.0,0,240.0,242.0,510.0,0.0,0.0,1,6,0
7102,0,0,2,21.0,0,86.0,3.0,149.0,208.0,329.0,0,5,0
7103,1,0,2,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0,0
7104,1,0,2,32.0,0,1.0,1146.0,0.0,50.0,34.0,0,0,0
7105,1,1,2,30.0,0,0.0,3208.0,0.0,2.0,330.0,1,0,0
7106,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,0,0
7107,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0,6,1
7108,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,6,1
7109,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0,4,1
7110,1,0,2,44.0,0,126.0,4688.0,0.0,0.0,12.0,1,4,1


In [394]:
flt=pd.DataFrame([[] for i in range(f_train_data.shape[0])])
for column in f_train_data.columns:
  if f_train_data[column].dtype=='float64':
    flt[column]=f_train_data[column]
flt.shape

(7111, 6)

In [395]:
columns=flt.columns
scaler=StandardScaler()
flt=pd.DataFrame(scaler.fit_transform(flt), columns=columns)
flt.shape

(7111, 6)

In [396]:
flt.tail(10)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
7101,-0.196652,0.024100,-0.141471,0.531884,-0.276749,-0.268902
7102,-0.540338,-0.211453,-0.283857,-0.044907,-0.092352,0.023841
7103,-1.983822,-0.342995,-0.285644,-0.282973,-0.276749,-0.268902
7104,0.215772,-0.341466,0.397094,-0.282973,-0.232422,-0.238649
7105,0.078297,-0.342995,1.625546,-0.282973,-0.274976,0.024730
7106,0.834408,-0.342995,3.776827,-0.282973,1.179811,-0.203057
7107,-0.746550,-0.342995,-0.285644,-0.282973,-0.276749,-0.268902
7108,-0.196652,-0.342995,-0.285644,2.708032,-0.275862,-0.268902
7109,0.215772,-0.342995,0.339306,-0.282973,0.036194,2.609590
7110,1.040620,-0.150270,2.507267,-0.282973,-0.276749,-0.258225


In [397]:
for column in flt.columns:
  f_train_data[column]=flt[column]

In [398]:
f_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7111 entries, 0 to 7110
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    7111 non-null   int64  
 1   CryoSleep     7111 non-null   int64  
 2   Destination   7111 non-null   int64  
 3   Age           7111 non-null   float64
 4   VIP           7111 non-null   int64  
 5   RoomService   7111 non-null   float64
 6   FoodCourt     7111 non-null   float64
 7   ShoppingMall  7111 non-null   float64
 8   Spa           7111 non-null   float64
 9   VRDeck        7111 non-null   float64
 10  Transported   7111 non-null   int64  
 11  Deck          7111 non-null   int64  
 12  Side          7111 non-null   int64  
dtypes: float64(6), int64(7)
memory usage: 722.3 KB


In [399]:
# vectorizer=TfidfVectorizer()

In [400]:
# lst=[]
# for column in f_train_data.columns:
#   if f_train_data[column].dtype=='O':
#     f_train_data[column]=f_train_data[column].apply(lambda x: re.sub("[^a-zA-Z]", "", x))
#     vectorizer.fit(f_train_data[column].values)
#     vector=vectorizer.transform(f_train_data[column])
#     vector=vector.toarray()
#     lst.append(vector)   
# lst=np.array(lst) 

In [401]:
# arr=[]
# for i in range(lst.shape[0]):
#   sub_arr=[]
#   for j in range(lst.shape[1]):
#     val=''
#     for k in range(lst.shape[2]):
#       val=val+str(int(lst[i][j][k]))
#     sub_arr.append(int(val))
#   arr.append(sub_arr)

In [402]:
# for sub_arr in range(len(arr)):
#   arr[sub_arr]=[int(j) for j in arr[sub_arr]]

In [403]:
# cout=0
# while(cout<len(arr)):
#   for column in f_train_data.columns:
#     if f_train_data[column].dtype=='O':
#       f_train_data[column]=arr[cout]
#       break
#   cout=cout+1

In [404]:
f_train_data.tail(10)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
7101,0,0,2,-0.196652,0,0.024100,-0.141471,0.531884,-0.276749,-0.268902,1,6,0
7102,0,0,2,-0.540338,0,-0.211453,-0.283857,-0.044907,-0.092352,0.023841,0,5,0
7103,1,0,2,-1.983822,0,-0.342995,-0.285644,-0.282973,-0.276749,-0.268902,1,0,0
7104,1,0,2,0.215772,0,-0.341466,0.397094,-0.282973,-0.232422,-0.238649,0,0,0
7105,1,1,2,0.078297,0,-0.342995,1.625546,-0.282973,-0.274976,0.024730,1,0,0
7106,1,0,0,0.834408,1,-0.342995,3.776827,-0.282973,1.179811,-0.203057,0,0,0
7107,0,1,1,-0.746550,0,-0.342995,-0.285644,-0.282973,-0.276749,-0.268902,0,6,1
7108,0,0,2,-0.196652,0,-0.342995,-0.285644,2.708032,-0.275862,-0.268902,1,6,1
7109,1,0,0,0.215772,0,-0.342995,0.339306,-0.282973,0.036194,2.609590,0,4,1
7110,1,0,2,1.040620,0,-0.150270,2.507267,-0.282973,-0.276749,-0.258225,1,4,1


In [405]:
x=f_train_data.drop(['Transported','Age','CryoSleep'],axis=1)
y=f_train_data['Transported']

In [406]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [407]:
model=RandomForestClassifier()

In [408]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [409]:
model.score(x_test,y_test)

0.8011243851018974

In [410]:
from xgboost import XGBClassifier
model = XGBClassifier()

In [411]:
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.8123682361208714